#### Exercise 5.4
Write PySpark code that will return the result of the following code block without using a left anti-join:  

left.join(right, how="left_anti", on="my_column").select("my_column").distinct()
  

In [0]:
left.join(right, how="left", 
          on=left["my_column"] == right["my_column"]
        ).where(right["my_column"].isnull()
).select(left["my_column"])

#### Exercise 5.5
Using the data from the data/broadcast_logs/Call_Signs.csv (careful: the delimiter here is the comma, not the pipe!), add the Undertaking_Name to our final table to display a human-readable description of the channel.

In [0]:
path_call_signs = ("/Volumes/workspace/dataanalysispysparkbook/bronze_files/eda/Call_Signs.csv")

In [0]:
import pyspark.sql.functions as F

df_call_signs = (spark.read.format("csv")
                            .option("sep", ",")
                            .option("header", "True")
                            .option("inferSchema", "True")
                            .load(path_call_signs)
)
df_call_signs.show(5, False)

df_human_readable = df_call_signs.drop("UndertakingNO")
df_human_readable.show(5, False)
                                         

#### Exercise 5.6
The government of Canada is asking for your analysis, but they’d like the PRC to be weighted differently. They’d like each PRC second to be considered 0.75 commercial seconds. Modify the program to account for this change.

In [0]:
PRC_vs_Commercial = (
    F.when(
            F.trim(F.col("ProgramClassCD")).isin(
                ["COM", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]
            ),
            F.col("duration_seconds"),
    )
    .when(
            F.trim(F.col("ProgramClassCD")) == "PRC",
            F.col("duration_seconds") * 0.75,
    )
    .otherwise(0)
)

exo5_6_df = (
    full_log.groupby("LogIdentifierID")
        .agg(
                F.sum(PRC_vs_Commercial).alias("duration_commercial"),
                F.sum("duration_seconds").alias("duration_total"),
        )
        .withColumn(
            "commercial_ratio",
            F.col("duration_commercial") / F.col("duration_total"),
        )
)

exo5_6_df.orderBy("commercial_ratio", ascending=False).show(5, False)

#### Exercise 5.7
On the data frame returned from commercials.py, return the number of channels in each bucket based on their commercial_ratio. (Hint: look at the documentation for round on how to round a value.)

In [0]:
exo5_7_df = (
    answer
        .groupby(F.round(F.col("commercial_ratio"),1).alias("commercial_ratio"))
        .agg(F.count("*").alias("number_of_channels"))
)